# 快速开始

介绍如何使用 nwpc-data 库。

首先导入需要使用的一些库。

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import eccodes

## 获取业务系统产品文件路径

可以使用内置的配置文件获取 CMA-PI 或二级存储中业务系统产品文件路径。

下面的示例获取 GRAPES GFS GMF 系统 2020 年 4 月 22 日 00 时次 024 时效的原始分辨率 GRIB 2 文件路径。

In [3]:
from nwpc_data.data_finder import find_local_file

file_path = find_local_file(
    "grapes_gfs_gmf/grib2/orig",
    start_time="2020042200",
    forecast_time="24h",
)
file_path

PosixPath('/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GFS_GMF/Prod-grib/2020042200/ORIG/gmf.gra.2020042200024.grb2')

使用 `number` 参数指定集合预报的成员编号。

下面示例获取 GRAPES GEPS 系统第 20 个成员 2020 年 4 月 22 日 12 时次 048 时效的原始分辨率 GRIB 2 文件路径。

In [4]:
file_path = find_local_file(
    "grapes_geps/grib2/orig",
    start_time=pd.to_datetime("2020-04-22 12:00:00"),
    forecast_time=pd.Timedelta(hours=48),
    number=20,
)
file_path

PosixPath('/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GEPS/Prod-grib/2020042212/grib2/gef.gra.020.2020042212048.grb2')

nwpc-data 还内置了共享数据集的部分数据。

获取 FNL 资料。

In [5]:
file_path = find_local_file(
    "glob/fnl",
    start_time="2019122000",
    data_class="cm",
)
file_path

PosixPath('/sstorage1/COMMONDATA/glob/fnl/2019/fnl_20191220_00_00.grib2')

内置的配置文件请查看 `nwpc_data/data_finder/conf` 目录

## 检索 GRIB 2 消息

nwpc-data 使用 [eccodes-python](https://github.com/ecmwf/eccodes-python) 从 GRIB 2 文件中检索需要的要素场，并返回 GRIB 2 消息。

下面的示例从文件中检索 850hPa 温度场。

In [6]:
data_path = find_local_file(
    "grapes_gfs_gmf/grib2/orig",
    start_time="2020042200",
    forecast_time="24h",
)
data_path

PosixPath('/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GFS_GMF/Prod-grib/2020042200/ORIG/gmf.gra.2020042200024.grb2')

In [7]:
from nwpc_data.grib.eccodes import load_message_from_file

t850 = load_message_from_file(
    data_path,
    parameter="t",
    level_type="pl",
    level=850,
)
t850

94506633934064

可以使用 eccodes-python 的 API 接口获取 GRIB 2 消息的属性和数据。

In [8]:
print("shortName:   ", eccodes.codes_get(t850, "shortName"))
print("typeOfLevel: ", eccodes.codes_get(t850, "typeOfLevel"))
print("level:       ", eccodes.codes_get(t850, "level"))

shortName:    t
typeOfLevel:  isobaricInhPa
level:        850


In [9]:
values = eccodes.codes_get_double_array(t850, "values")
values = values.reshape([720, 1440])
values

array([[258.75289063, 258.76289063, 258.76289063, ..., 258.75289063,
        258.76289063, 258.75289063],
       [258.47289062, 258.48289062, 258.46289062, ..., 258.46289062,
        258.47289062, 258.44289063],
       [258.27289063, 258.26289063, 258.27289063, ..., 258.25289063,
        258.26289063, 258.26289063],
       ...,
       [227.68289062, 227.68289062, 227.72289062, ..., 227.62289062,
        227.75289062, 227.63289063],
       [227.90289062, 227.90289062, 227.90289062, ..., 227.90289062,
        227.88289063, 227.91289063],
       [228.12289062, 228.18289062, 228.19289063, ..., 228.09289062,
        228.23289063, 228.20289063]])

**注意**：需要手动调用 `eccodes.codes_release` 释放消息对象。

In [10]:
eccodes.codes_release(t850)

## 加载 GRIB 2 要素场

nwpc-data 还提供对上述检索得到 GRIB 2 消息的封装，返回 `xarray.DataArray` 对象，类似 [cfgrib](https://github.com/ecmwf/cfgrib) 库。

In [11]:
from nwpc_data.grib.eccodes import load_field_from_file

t850 = load_field_from_file(
    data_path,
    parameter="t",
    level_type="pl",
    level=850,
)
t850

<xarray.DataArray 't' (latitude: 720, longitude: 1440)>
array([[258.75289063, 258.76289063, 258.76289063, ..., 258.75289063,
        258.76289063, 258.75289063],
       [258.47289062, 258.48289062, 258.46289062, ..., 258.46289062,
        258.47289062, 258.44289063],
       [258.27289063, 258.26289063, 258.27289063, ..., 258.25289063,
        258.26289063, 258.26289063],
       ...,
       [227.68289062, 227.68289062, 227.72289062, ..., 227.62289062,
        227.75289062, 227.63289063],
       [227.90289062, 227.90289062, 227.90289062, ..., 227.90289062,
        227.88289063, 227.91289063],
       [228.12289062, 228.18289062, 228.19289063, ..., 228.09289062,
        228.23289063, 228.20289063]])
Coordinates:
    time        datetime64[ns] 2020-04-22
    step        timedelta64[ns] 1 days
    valid_time  datetime64[ns] 2020-04-23
    pl          float64 850.0
  * latitude    (latitude) float64 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Attributes:
    GRIB_edition:                    2
    GRIB_centre:                     babj
    GRIB_subCentre:                  0
    GRIB_tablesVersion:              4
    GRIB_localTablesVersion:         1
    GRIB_dataType:                   fc
    GRIB_dataDate:                   20200422
    GRIB_dataTime:                   0
    GRIB_validityDate:               20200423
    GRIB_validityTime:               0
    GRIB_step:                       24
    GRIB_stepType:                   instant
    GRIB_stepUnits:                  1
    GRIB_stepRange:                  24
    GRIB_endStep:                    24
    GRIB_name:                       Temperature
    GRIB_shortName:                  t
    GRIB_cfName:                     air_temperature
    GRIB_discipline:                 0
    GRIB_parameterCategory:          0
    GRIB_parameterNumber:            0
    GRIB_gridType:                   regular_ll
    GRIB_gridDefinitionDescription:  Latitude/longitude 
    GRIB_typeOfFirstFixedSurface:    pl
    GRIB_typeOfLevel:                isobaricInhPa
    GRIB_level:                      850
    GRIB_numberOfPoints:             1036800
    GRIB_missingValue:               9999
    GRIB_units:                      K
    long_name:                       Temperature
    units:                           K

`t850` 对象已对 GRIB 2 消息进行解码，包含两个维度：纬度（`latitude`）和经度（`longitude`）。

可以使用 xarray 库提供的一系列工具对数据进行处理。比如，求纬向平均值。

In [12]:
t850.mean(dim="longitude")

<xarray.DataArray 't' (latitude: 720)>
array([258.89689063, 258.92158507, 258.9702934 , 259.03200868,
       259.13369618, 259.25227951, 259.36137674, 259.43795313,
       259.48117535, 259.51569618, 259.54527951, 259.58036285,
       259.61993229, 259.66970313, 259.73472396, 259.8143559 ,
       259.90958507, 260.01493924, 260.12211979, 260.23601562,
       260.33834201, 260.43259896, 260.52713368, 260.62855035,
       260.73437674, 260.8412934 , 260.95026562, 261.07211979,
       261.20684201, 261.33564757, 261.47109201, 261.61936979,
       261.77783507, 261.94189062, 262.09409896, 262.24932812,
       262.38886979, 262.47721007, 262.54587674, 262.62440451,
       262.69916146, 262.75284201, 262.78774479, 262.80559896,
       262.81419618, 262.82962674, 262.84596007, 262.86258507,
       262.88671701, 262.89856424, 262.91805729, 262.93743229,
       262.96751563, 262.98624479, 262.98996701, 262.9832309 ,
       262.97455729, 262.96696701, 262.95727257, 262.9509184 ,
       262.94531424, 262.92838368, 262.92327257, 262.90801562,
       262.89346701, 262.89584896, 262.89827257, 262.92536979,
       262.95557118, 262.98227951, 263.00800174, 263.05213368,
       263.1318559 , 263.24283507, 263.35055729, 263.44265451,
       263.53114757, 263.61167535, 263.69661285, 263.76896007,
...
       256.6772934 , 256.13555729, 255.62593924, 255.03613368,
       254.40217535, 253.76339757, 253.17752257, 252.57847396,
       252.08197396, 251.61507813, 251.16218229, 250.72641146,
       250.3174809 , 249.88568229, 249.41555729, 248.93059201,
       248.48537674, 248.1347934 , 247.8771059 , 247.66580035,
       247.41715451, 247.12427951, 246.84252257, 246.5109809 ,
       246.13997396, 245.80788368, 245.58077951, 245.36372396,
       245.09222396, 244.80211285, 244.54525868, 244.31832812,
       244.13328646, 243.99023785, 243.96364063, 243.99811285,
       244.0030434 , 243.93723785, 243.86811979, 243.77822396,
       243.68809896, 243.59026563, 243.45472396, 243.28675174,
       243.03309201, 242.71301562, 242.35030035, 241.9659184 ,
       241.62101563, 241.32216146, 240.97146701, 240.57764757,
       240.1086059 , 239.71820313, 239.40714062, 239.18649479,
       238.92576563, 238.73946701, 238.40955035, 237.87097396,
       237.31382812, 236.76059896, 235.83973785, 235.16499479,
       234.68525868, 234.24601563, 233.83736285, 233.55559896,
       233.21757118, 232.8665434 , 232.40089062, 231.89748785,
       231.30739757, 230.7532309 , 230.25101563, 229.83740451,
       229.42580035, 229.02872396, 228.65664757, 228.51356424])
Coordinates:
    time        datetime64[ns] 2020-04-22
    step        timedelta64[ns] 1 days
    valid_time  datetime64[ns] 2020-04-23
    pl          float64 850.0
  * latitude    (latitude) float64 89.88 89.62 89.38 ... -89.38 -89.62 -89.88

## 下一步

请浏览下一章节 [介绍和安装](./01_introduction.ipynb)。